In [2]:
#以這個為主
import time
import datetime
import queue
import pymysql
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen

t = time.time()
myq = queue.Queue()

def getOID(strword):
    s1 = "."
    strword = strword.replace("SNMPv2-SMI::mib-2", "1.3.6.1.2.1")
    arr = strword.split(".")
    del arr[-1]
    return s1.join(arr)

# ip lists 最後改撈資料庫
# 建立MySQL連線
dbBase = pymysql.connect(host='140.134.51.26', port=3306, user='root', passwd='Gtml@3777', db='ResourceApp', charset='utf8')

# # 電腦設備IP
# cursor = dbBase.cursor()
# def listPC(cursor):
#     sqlIp = "SELECT IPAddress1 FROM ResourceApp.DeviceConf where IsOnline = 1"
#     cursor.execute(sqlIp)
#     resultIp = cursor.fetchall()
#     return resultIp

# resultIp = listPC(cursor)
# hosts = []
# for row in resultIp:
#     # print("Put IP:", row[0])
#     hosts.append(row[0])

# 建立MIB對應表
cursor = dbBase.cursor()
sql = "SELECT Oid, OidName FROM ResourceApp.MIBConf"
cursor.execute(sql)
mibResults = cursor.fetchall()
mibResults = list(mibResults)
mibResults = dict(mibResults)

# 创建 cmd generator
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

#串資料
# snmpJson = {}
# snmpJson["ip"] = host
nowDate = datetime.datetime.now()
# snmpJson["receiveTime"] = nowDate
listIPs = []

#回调函数。在有数据返回时触发
def cbFun(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
#     hrMemorySize = 0
    if cbCtx[1] not in listIPs:
        print("cbCtx[2]IP:", cbCtx[2]["ip"])
        cbCtx[2]["receiveTime"] = nowDate
        system = []
        ifTable = {}
        ifIndex = []
        ifDescr = []
        ifType = []
        ifMtu = []
        ifSpeed = []
        ifPhysAddress = []
        ifAdminStatus = []
        ifOperStatus = []
        ifLastChange = []
        ifInOctets = []
        ifInUcastPkts = []
        ifInNUcastPkts = []
        ifInDiscards = []
        ifInErrors = []
        ifInUnknownProtos = []
        ifOutOctets = []
        ifOutUcastPkts = []
        ifOutNUcastPkts = []
        ifOutDiscards = []
        ifOutErrors = []
        ifOutQLen = []
        ifSpecific = []

        hrSystem = []

        hrStorage = {}
        hrMemorySize = 0
        hrStorageIndex = []
        hrStorageType = []
        hrStorageDescr = []
        hrStorageAllocationUnits = []
        hrStorageSize = []
        hrStorageUsed = []
        hrStorageAllocationFailures = []

        hrSWRun = {}
        hrSWRunIndex = []
        hrSWRunName = []
        hrSWRunID = []
        hrSWRunPath = []
        hrSWRunParameters = []
        hrSWRunType = []
        hrSWRunStatus = []
        hrSWRunPerfCPU = []
        hrSWRunPerfMem = []
        hrSWInstalled = {}
        hrSWInstalledIndex = []
        hrSWInstalledName = []
        hrSWInstalledID = []
        hrSWInstalledType = []
        hrSWInstalledDate = []
        cbCtx[2]["system"] = system
        ifTable["ifIndex"] = ifIndex
        ifTable["ifDescr"] = ifDescr
        ifTable["ifType"] = ifType
        ifTable["ifMtu"] = ifMtu
        ifTable["ifSpeed"] = ifSpeed
        ifTable["ifPhysAddress"] = ifPhysAddress
        ifTable["ifAdminStatus"] = ifAdminStatus
        ifTable["ifOperStatus"] = ifOperStatus
        ifTable["ifLastChange"] = ifLastChange
        ifTable["ifInOctets"] = ifInOctets
        ifTable["ifInUcastPkts"] = ifInUcastPkts
        ifTable["ifInNUcastPkts"] = ifInNUcastPkts
        ifTable["ifInDiscards"] = ifInDiscards
        ifTable["ifInErrors"] = ifInErrors
        ifTable["ifInUnknownProtos"] = ifInUnknownProtos
        ifTable["ifOutOctets"] = ifOutOctets
        ifTable["ifOutUcastPkts"] = ifOutUcastPkts
        ifTable["ifOutNUcastPkts"] = ifOutNUcastPkts
        ifTable["ifOutDiscards"] = ifOutDiscards
        ifTable["ifOutErrors"] = ifOutErrors
        ifTable["ifOutQLen"] = ifOutQLen
        ifTable["ifSpecific"] = ifSpecific
        cbCtx[2]["ifTable"] = ifTable
        cbCtx[2]["hrSystem"] = hrSystem
        hrStorage["hrMemorySize"] = hrMemorySize
        hrStorage["hrStorageIndex"] = hrStorageIndex
        hrStorage["hrStorageType"] = hrStorageType
        hrStorage["hrStorageDescr"] = hrStorageDescr
        hrStorage["hrStorageAllocationUnits"] = hrStorageAllocationUnits
        hrStorage["hrStorageSize"] = hrStorageSize
        hrStorage["hrStorageUsed"] = hrStorageUsed
        hrStorage["hrStorageAllocationFailures"] = hrStorageAllocationFailures
        cbCtx[2]["hrStorage"] = hrStorage
        hrSWRun["hrSWRunIndex"] = hrSWRunIndex
        hrSWRun["hrSWRunName"] = hrSWRunName
        hrSWRun["hrSWRunID"] = hrSWRunID
        hrSWRun["hrSWRunPath"] = hrSWRunPath
        hrSWRun["hrSWRunParameters"] = hrSWRunParameters
        hrSWRun["hrSWRunType"] = hrSWRunType
        hrSWRun["hrSWRunStatus"] = hrSWRunStatus
        hrSWRun["hrSWRunPerfCPU"] = hrSWRunPerfCPU
        hrSWRun["hrSWRunPerfMem"] = hrSWRunPerfMem
        cbCtx[2]["hrSWRun"] = hrSWRun
        hrSWInstalled["hrSWInstalledIndex"] = hrSWInstalledIndex
        hrSWInstalled["hrSWInstalledName"] = hrSWInstalledName
        hrSWInstalled["hrSWInstalledID"] = hrSWInstalledID
        hrSWInstalled["hrSWInstalledType"] = hrSWInstalledType
        hrSWInstalled["hrSWInstalledDate"] = hrSWInstalledDate
        cbCtx[2]["hrSWInstalled"] = hrSWInstalled
        listIPs.append(cbCtx[1])

    myq.put((time.time()-t, varBinds, errorIndication, errorStatus, errorIndex, cbCtx))
#     print("IPP:", cbCtx[1])
#     snmpJson["ip"] = cbCtx[1]
    if errorIndication:  # SNMP agent errors
        print("errorIndication:", errorIndication)
        return 0  # 告诉 walk 停止运行
    else:
        if errorStatus:
            print('%s at %s' % (errorStatus.prettyPrint(), varBinds[int(errorIndex)-1] if errorIndex else '?'))
        else:
            for row in varBinds:
                for oid, val in row:
#                     print("cbCtx[0]:", cbCtx[0], "oid:", oid.prettyPrint())
                    if cbCtx[0] not in oid.prettyPrint():
#                         print(snmpJson)
#                         print(result[len(result)-1])
                        print("IP:", cbCtx[1],"cbCtx[0]:", cbCtx[0], "oid:", oid.prettyPrint())
                        print("===================")
#                       Linux: cbCtx[0]: 1.3.6.1.2.1.25.6 oid: 1.3.6.1.4.1.2021.4.1.0
#                       Windows: cbCtx[0]: 1.3.6.1.2.1.25.6 oid: 1.3.6.1.2.1.31.1.1.1.1.1
                        if cbCtx[0] == "1.3.6.1.2.1.25.6" and oid.prettyPrint() == "1.3.6.1.2.1.31.1.1.1.1.1":
                            print(cbCtx[2])
                        return 0  # 告诉 walk 停止运行
                    
                    snmpContext = {}
                    snmpContext["oid"] = oid.prettyPrint()
                    snmpContext["val"] = val.prettyPrint()
                    oidKey = getOID(oid.prettyPrint())
                    snmpContext["oid"] = oidKey
                    snmpContext["val"] = val.prettyPrint()
                    oidName = mibResults[cbCtx[0]]
                    if "system" == oidName:
                        cbCtx[2]["system"].append(snmpContext)
                    if "ifTable" == oidName:
                        oidValue = mibResults[oidKey]
                        if "ifIndex" == oidValue :
                            cbCtx[2]["ifTable"]["ifIndex"].append(val.prettyPrint())
                        if "ifDescr" == oidValue :
                            cbCtx[2]["ifTable"]["ifDescr"].append(val.prettyPrint())
                        if "ifType" == oidValue :
                            cbCtx[2]["ifTable"]["ifType"].append(val.prettyPrint())
                        if "ifMtu" == oidValue :
                            cbCtx[2]["ifTable"]["ifMtu"].append(val.prettyPrint())
                        if "ifSpeed" == oidValue :
                            cbCtx[2]["ifTable"]["ifSpeed"].append(val.prettyPrint())
                        if "ifPhysAddress" == oidValue :
                            cbCtx[2]["ifTable"]["ifPhysAddress"].append(val.prettyPrint())
                        if "ifAdminStatus" == oidValue :
                            cbCtx[2]["ifTable"]["ifAdminStatus"].append(val.prettyPrint())
                        if "ifOperStatus" == oidValue :
                            cbCtx[2]["ifTable"]["ifOperStatus"].append(val.prettyPrint())
                        if "ifLastChange" == oidValue :
                            cbCtx[2]["ifTable"]["ifLastChange"].append(val.prettyPrint())
                        if "ifInOctets" == oidValue :
                            cbCtx[2]["ifTable"]["ifInOctets"].append(val.prettyPrint())
                        if "ifInUcastPkts" == oidValue :
                            cbCtx[2]["ifTable"]["ifInUcastPkts"].append(val.prettyPrint())
                        if "ifInNUcastPkts" == oidValue :
                            cbCtx[2]["ifTable"]["ifInNUcastPkts"].append(val.prettyPrint())
                        if "ifInDiscards" == oidValue :
                            cbCtx[2]["ifTable"]["ifInDiscards"].append(val.prettyPrint())
                        if "ifInErrors" == oidValue :
                            cbCtx[2]["ifTable"]["ifInErrors"].append(val.prettyPrint())
                        if "ifInUnknownProtos" == oidValue :
                            cbCtx[2]["ifTable"]["ifInUnknownProtos"].append(val.prettyPrint())
                        if "ifOutOctets" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutOctets"].append(val.prettyPrint())
                        if "ifOutUcastPkts" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutUcastPkts"].append(val.prettyPrint())
                        if "ifOutNUcastPkts" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutNUcastPkts"].append(val.prettyPrint())
                        if "ifOutDiscards" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutDiscards"].append(val.prettyPrint())
                        if "ifOutErrors" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutErrors"].append(val.prettyPrint())
                        if "ifOutQLen" == oidValue :
                            cbCtx[2]["ifTable"]["ifOutQLen"].append(val.prettyPrint())
                        if "ifSpecific" == oidValue :
                            cbCtx[2]["ifTable"]["ifSpecific"].append(val.prettyPrint())
                    if "hrSystem" == oidName:
                        cbCtx[2]["hrSystem"].append(snmpContext)
                    if "hrStorage" == oidName:
                        oidValue = mibResults[oidKey]
                        if "hrMemorySize" == oidValue :
                            cbCtx[2]["hrStorage"]["hrMemorySize"] = val.prettyPrint()
                        if "hrStorageIndex" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageIndex"].append(val.prettyPrint())
                        if "hrStorageType" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageType"].append(val.prettyPrint())
                        if "hrStorageDescr" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageDescr"].append(val.prettyPrint())
                        if "hrStorageAllocationUnits" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageAllocationUnits"].append(val.prettyPrint())
                        if "hrStorageSize" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageSize"].append(val.prettyPrint())
                        if "hrStorageUsed" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageUsed"].append(val.prettyPrint())
                        if "hrStorageAllocationFailures" == oidValue :
                            cbCtx[2]["hrStorage"]["hrStorageAllocationFailures"].append(val.prettyPrint())
                    if "hrSWRun" == oidName:
                        oidValue = mibResults[oidKey]
                        if "hrSWRunIndex" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunIndex"].append(val.prettyPrint())
                        if "hrSWRunName" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunName"].append(val.prettyPrint())
                        if "hrSWRunID" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunID"].append(val.prettyPrint())
                        if "hrSWRunPath" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunPath"].append(val.prettyPrint())
                        if "hrSWRunParameters" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunParameters"].append(val.prettyPrint())
                        if "hrSWRunType" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunType"].append(val.prettyPrint())
                        if "hrSWRunStatus" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunStatus"].append(val.prettyPrint())
                    if "hrSWRunPerf" == oidName:
                        oidValue = mibResults[oidKey]
                        if "hrSWRunPerfCPU" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunPerfCPU"].append(val.prettyPrint())
                        if "hrSWRunPerfMem" == oidValue :
                            cbCtx[2]["hrSWRun"]["hrSWRunPerfMem"].append(val.prettyPrint())
                    if "hrSWInstalled" == oidName:
                        oidValue = mibResults[oidKey]
                        if "hrSWInstalledIndex" == oidValue :
                            cbCtx[2]["hrSWInstalled"]["hrSWInstalledIndex"].append(val.prettyPrint())
                        if "hrSWInstalledName" == oidValue :
                            cbCtx[2]["hrSWInstalled"]["hrSWInstalledName"].append(val.prettyPrint())
                        if "hrSWInstalledID" == oidValue :
                            cbCtx[2]["hrSWInstalled"]["hrSWInstalledID"].append(val.prettyPrint())
                        if "hrSWInstalledType" == oidValue :
                            cbCtx[2]["hrSWInstalled"]["hrSWInstalledType"].append(val.prettyPrint())
                        if "hrSWInstalledDate" == oidValue :
                            cbCtx[2]["hrSWInstalled"]["hrSWInstalledDate"].append(val.prettyPrint())

            return 1  # 告诉 walk 继续运行
    

hosts = ["140.134.211.9","140.134.207.30","140.134.213.101"]
# hosts = ["140.134.207.30"]
# hosts = ["140.134.211.9","140.134.211.1","140.134.211.100","140.134.213.101","140.134.211.59"]
oids = ['1.3.6.1.2.1.1','1.3.6.1.2.1.2.2','1.3.6.1.2.1.25.1','1.3.6.1.2.1.25.2','1.3.6.1.2.1.25.4','1.3.6.1.2.1.25.5','1.3.6.1.2.1.25.6']
# oids = ["1.3.6.1.2.1.25.4"]


#添加任务 cmdgen.CommunityData('rmc2772'),
for h in hosts:
    print("host:", h)
    snmpJson = {}
    snmpJson["ip"] = h
    for oid in oids:
        print("O:", oid)
        g=cmdGen.nextCmd(
            cmdgen.CommunityData('rmc2772'),
            cmdgen.UdpTransportTarget((h, 161)),
            (cmdgen.MibVariable(oid),),
            (cbFun, (oid,h,snmpJson))
        )


time1 = time.time() - t

#执行异步获取snmp
cmdGen.snmpEngine.transportDispatcher.runDispatcher()

# 關閉連線
cursor.close()
dbBase.close()


#打印结果
# while True:
#     try:
#         info = myq.get(block=False)
#         ip = info[5][1]
#         print("IP:", info[5][1])
#         if info[2]:  # SNMP agent errors
#             print("errorIndication:", info[2])
#         else:
#             if info[3]:
#                 print("here")
#                 print('%s at %s' % (info[3].prettyPrint(), info[1][int(info[4])-1] if info[4] else '?'))
#             else:
#                 for row in info[1]:
#                     #print("row:", row)
# #                     oid, value = row[0]
# #                     print(oid, '=', value)
#                     for oid, val in row:
#                         print("oid:", getOID(oid.prettyPrint()), "val:", val.prettyPrint())
#                         if "ip" in snmpJson.keys():
#                             #######
#                         else:
#                             snmpJson["ip"] = ip
#     except queue.Empty:
#         print(time1)
#         print(time.time() - t)
#         break

host: 140.134.211.9
O: 1.3.6.1.2.1.1
O: 1.3.6.1.2.1.2.2
O: 1.3.6.1.2.1.25.1
O: 1.3.6.1.2.1.25.2
O: 1.3.6.1.2.1.25.4
O: 1.3.6.1.2.1.25.5
O: 1.3.6.1.2.1.25.6
host: 140.134.207.30
O: 1.3.6.1.2.1.1
O: 1.3.6.1.2.1.2.2
O: 1.3.6.1.2.1.25.1
O: 1.3.6.1.2.1.25.2
O: 1.3.6.1.2.1.25.4
O: 1.3.6.1.2.1.25.5
O: 1.3.6.1.2.1.25.6
host: 140.134.213.101
O: 1.3.6.1.2.1.1
O: 1.3.6.1.2.1.2.2
O: 1.3.6.1.2.1.25.1
O: 1.3.6.1.2.1.25.2
O: 1.3.6.1.2.1.25.4
O: 1.3.6.1.2.1.25.5
O: 1.3.6.1.2.1.25.6
cbCtx[2]IP: 140.134.213.101
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.1 oid: 1.3.6.1.2.1.2.1.0
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.25.1 oid: 1.3.6.1.2.1.25.2.2.0
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.25.2 oid: 1.3.6.1.2.1.25.3.2.1.1.1
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.25.5 oid: 1.3.6.1.2.1.25.6.1.0
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.25.6 oid: 1.3.6.1.2.1.31.1.1.1.1.1
{'ip': '140.134.213.101', 'receiveTime': datetime.datetime(2020, 7, 11, 20, 24, 38, 306829), 'system': [{'oid': '1.3.6.1.2.1.1.1', 'va

IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.2.2 oid: 1.3.6.1.2.1.4.1.0
IP: 140.134.213.101 cbCtx[0]: 1.3.6.1.2.1.25.4 oid: 1.3.6.1.2.1.25.5.1.1.1.1
cbCtx[2]IP: 140.134.211.9
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
cbCtx[2]IP: 140.134.207.30
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received before timeout
errorIndication: No SNMP response received befo